In [24]:

from importlib import reload
import plot, utils, data_load, clustering, constants 
import warnings
import random

# use reload whenever make changes to underlying file
# modules_to_reload = [plot, utils, data_load, clustering, constants ]

# for module in modules_to_reload:
#     reload(module)


In [11]:
# Suppress FutureWarnings from multiple modules
modules_to_ignore = ("seaborn", "sklearn")

for module in modules_to_ignore:
    warnings.filterwarnings("ignore", category=FutureWarning, module=module)

In [3]:
maf_df = data_load.load_maf_data(file_path=constants.maf_file_path)
expression_df = data_load.load_txt_file_into_dataframe(file_path=constants.expression_file_path)
expression_df_melted = data_load.reformat_expression_data(df=expression_df)
mutation_expression_df_melted = data_load.preprocess_and_combine_mutation_expression(maf_df= maf_df, expression_df = expression_df_melted)

fraction of rows filtered is 0.2865853658536585


In [19]:
from sklearn.metrics import silhouette_samples, silhouette_score

In [17]:
heatmap_data.T

gene,DUSP26,ART4,DNAJC14,CABP7,SAMD11,KIF3B,FAM118B,SH2D5,FAM104B,MARS1,...,ZNF681,FAM72C,TMTC3,CAVIN3,SIGLEC11,FOXE1,KIF18A,NR1D1,DNAJB7,H3C8
RESPOND_10100218,0.000000,2.321928,6.044394,2.000000e+00,3.169925,4.754888,3.807355,0.084359,3.700440,8.219169,...,0.540312,0.128897,8.144658,4.643856,0.894752,0.371567,0.648259,3.807355,0.770253,0.711070
RESPOND_10100291,1.584963,0.985656,6.614710,2.584963e+00,5.044394,5.169925,4.247928,1.000000,2.321928,8.076816,...,0.706748,0.130190,7.507795,4.807355,1.584963,2.000000,0.527582,3.321928,0.358442,0.367105
RESPOND_10100412,2.584963,2.000000,5.614710,1.584963e+00,6.022368,4.392317,2.807355,0.522905,2.321928,7.467606,...,0.351492,0.165748,7.238405,7.139551,0.500103,1.584963,1.000000,4.700440,0.666276,0.107996
RESPOND_10100478,0.510793,0.791744,6.169925,5.092732e-01,4.643856,4.754888,4.857981,0.370090,3.000000,7.888743,...,0.588161,0.079270,6.584963,6.149747,1.000000,0.018271,0.716525,3.700440,0.570646,0.900355
RESPOND_10100596,1.584963,1.584963,5.754888,1.584963e+00,4.906891,4.700440,3.000000,0.819593,2.000000,7.845490,...,0.128868,0.180467,6.375039,6.392317,1.000000,0.672833,0.925354,4.700440,0.498017,0.898959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RESPOND_80100556,1.584963,1.000000,6.339850,3.554335e-08,3.169925,5.169925,3.584963,0.000000,3.459432,6.357552,...,1.000000,0.000000,7.651052,5.672425,0.000000,0.457016,0.569096,5.832890,0.610468,2.807355
RESPOND_80100590,4.906891,2.807355,5.523562,2.321928e+00,3.459432,4.459432,3.459432,0.836079,2.000000,8.184875,...,0.272320,0.302931,5.129283,5.977280,1.584963,2.000000,0.000000,3.584963,1.000000,0.000000
RESPOND_81100031,1.584963,0.091864,5.459432,1.000000e+00,5.954196,2.584963,2.000000,1.000000,1.000000,7.294621,...,0.111361,0.000000,4.807355,7.228819,1.000000,2.321928,0.000000,4.807355,0.018179,0.244706
RESPOND_40100842,1.584963,1.584963,6.247928,6.472005e-01,4.807355,4.643856,3.459432,0.476445,3.321928,8.055282,...,0.947253,0.205287,5.906891,4.247928,0.689537,0.933109,0.292142,3.807355,1.000000,0.000000


In [18]:
col_clusters

{'Cluster 1': Index(['RESPOND_10100615', 'RESPOND_10101376', 'RESPOND_20100327',
        'RESPOND_20100346', 'RESPOND_20100373', 'RESPOND_20100487',
        'RESPOND_20100591', 'RESPOND_20100762', 'RESPOND_20101329',
        'RESPOND_20101348', 'RESPOND_20101765', 'RESPOND_30100563',
        'RESPOND_40100063', 'RESPOND_40100194', 'RESPOND_40100476',
        'RESPOND_40100584', 'RESPOND_40100674', 'RESPOND_50100020',
        'RESPOND_50100788', 'RESPOND_50100954', 'RESPOND_60100201',
        'RESPOND_60100256', 'RESPOND_60100337', 'RESPOND_60100355',
        'RESPOND_60100564', 'RESPOND_60100604', 'RESPOND_60100794',
        'RESPOND_60100876', 'RESPOND_60100883', 'RESPOND_60100937',
        'RESPOND_60101324', 'RESPOND_60101373', 'RESPOND_60102354',
        'RESPOND_61100294', 'RESPOND_61100364', 'RESPOND_61100480',
        'RESPOND_61100487', 'RESPOND_61100497', 'RESPOND_70100042',
        'RESPOND_70100164', 'RESPOND_70100286', 'RESPOND_70100393',
        'RESPOND_70100743', 'RESPON

In [41]:

reload(clustering)
reload(utils)
reload(data_load)
scores_genes = {}
for target_gene in constants.genes:  
    # target_gene = random.choice(constants.genes). use when testing

    # calculate logfc, pvalue
    volcano_plot_df, individuals_mutated_target_gene, volcano_input_filename = utils.generate_stats_per_gene(
        express_mut_genes_df=mutation_expression_df_melted, 
        target_gene=target_gene,
        output_folder=target_gene)

    # get top 100 genes with the lowest p-values
    heatmap_data = data_load.generate_expression_heatmap(expression_df=expression_df, volcano_plot_df=volcano_plot_df, n=100)

    # cluster top 100 genes and samples
    # evaluate clustering with sihlouttee 
    # filter to only get silhouttee score for mutatted samples
    row_linkage, col_linkage, row_clusters, col_clusters, row_score, col_score, mutated_col_score = clustering.hierarchical_clustering(expression_df_heatmap=heatmap_data, 
                                                                                                output_folder=target_gene,
                                                                                                row_threshold=7, 
                                                                                                col_threshold=7,
                                                                                                mutated_samples=individuals_mutated_target_gene)
    

    scores_genes[target_gene]={'gene_sil_score': row_score, 'sample_sil_score': col_score, 'mutated_sample_sil_score': mutated_col_score}



outputting data to TRPM1/6_111_logfc_pvalue.csv
Row Silhouette Score: 0.1645550843863594
Column Silhouette Score: 0.02277090086582008

Individual Silhouette Scores for Mutated Samples:
Mutated Sample RESPOND_10100412: 0.33902993555397104
Mutated Sample RESPOND_20100780: 0.39358489068079333
Mutated Sample RESPOND_20101031: 0.38635628727563553
Mutated Sample RESPOND_40100031: 0.23069345858697174
Mutated Sample RESPOND_61100254: 0.3113567675370003
Mutated Sample RESPOND_80100170: 0.34805086936731455

Average Silhouette Score for Mutated Samples: 0.3348453681669478


In [ ]:
# # getting indviduals with mutation for heatmap
# # TODO: combine this func into other functions
# mutated_status_df = utils.get_mutated_status(expression_df_heatmap=heatmap_data, 
#                                             individuals_mutated_target_gene=individuals_mutated_target_gene, 
#                                             output_folder=target_gene)

# # plot clustered heatmap
# clustered_heatmap = plot.create_clustered_heatmap_and_save(heatmap_data, 
#                                                       row_linkage, 
#                                                       col_linkage,
#                                                       mutated_status_df,
#                                                       output_folder=target_gene)

# # plot truncated dendograms 
# row_dendrogram, col_dendrogram = clustering.plot_and_save_dendrograms(row_linkage, 
#                                                            col_linkage, 
#                                                            heatmap_data, 
#                                                            output_folder=target_gene)

# # plot histograms of adjusted and regular pvalues 
# plot.histogram_of_column_and_save(volcano_plot_df, 'pvalue', output_folder=target_gene)
# plot.histogram_of_column_and_save(volcano_plot_df, 'adjusted_pvalue', output_folder=target_gene)

# # calculate significant genes determined by cutoffs
# significant_genes_positive, significant_genes_negative  = plot.volcano_plot(input_file_path=volcano_input_filename, 
#                                                                         yaxis='pvalue', 
#                                                                         xaxis='logFC', 
#                                                                         output_folder=target_gene,
#                                                                         significance_threshold=0.05, 
#                                                                         logfold_positive_threshold=2, 
#                                                                         logfold_negative_threshold=-2)

# # create box plots of significant genes (uoregulated and downregulated genes)
# plot.create_gene_expression_boxplot(expression_df, significant_genes_positive, mutated_status_df, positive=1, output_folder=target_gene)
# plot.create_gene_expression_boxplot(expression_df, significant_genes_negative, mutated_status_df, positive=0, output_folder=target_gene)

# # Reset warnings to default behavior after the code block
# warnings.resetwarnings()

In [ ]:
import plot, utils, data_load, clustering, constants 
import warnings
import random


# Suppress FutureWarnings from multiple modules
modules_to_ignore = ("seaborn", "sklearn")

for module in modules_to_ignore:
    warnings.filterwarnings("ignore", category=FutureWarning, module=module)

maf_df = data_load.load_maf_data(file_path=constants.maf_file_path)
expression_df = data_load.load_txt_file_into_dataframe(file_path=constants.expression_file_path)
expression_df_melted = data_load.reformat_expression_data(df=expression_df)
mutation_expression_df_melted = data_load.preprocess_and_combine_mutation_expression(maf_df= maf_df, expression_df = expression_df_melted)

sil_scores_genes = {}
for target_gene in constants.genes:
    # calculate logfc, pvalue
    volcano_plot_df, individuals_mutated_target_gene, volcano_input_filename = utils.generate_stats_per_gene(
        express_mut_genes_df=mutation_expression_df_melted, 
        target_gene=target_gene,
        output_folder=target_gene)

    # get top 100 genes with the lowest p-values
    heatmap_data = data_load.generate_expression_heatmap(expression_df=expression_df, volcano_plot_df=volcano_plot_df, n=100)

    mutated_status_df = utils.get_mutated_status(expression_df_heatmap=heatmap_data, 
                                             individuals_mutated_target_gene=individuals_mutated_target_gene, 
                                             output_folder=target_gene)

    # cluster top 100 genes and samples
    row_linkage, col_linkage, row_clusters, col_clusters, gene_sil_score, sample_sil_score = clustering.hierarchical_clustering(
                                                                                                expression_df_heatmap=heatmap_data, 
                                                                                                output_folder=target_gene,
                                                                                                row_threshold=7, 
                                                                                                col_threshold=2)

    sil_scores_genes[target_gene]=gene_sil_score


print(sil_scores_genes)

# # getting indviduals with mutation for heatmap
# # TODO: combine this func into other functions
# mutated_status_df = utils.get_mutated_status(expression_df_heatmap=heatmap_data, 
#                                             individuals_mutated_target_gene=individuals_mutated_target_gene, 
#                                             output_folder=target_gene)

# # plot clustered heatmap
# clustered_heatmap = plot.create_clustered_heatmap_and_save(heatmap_data, 
#                                                       row_linkage, 
#                                                       col_linkage,
#                                                       mutated_status_df,
#                                                       output_folder=target_gene)

# # plot truncated dendograms 
# row_dendrogram, col_dendrogram = clustering.plot_and_save_dendrograms(row_linkage, 
#                                                            col_linkage, 
#                                                            heatmap_data, 
#                                                            output_folder=target_gene)

# # plot histograms of adjusted and regular pvalues 
# plot.histogram_of_column_and_save(volcano_plot_df, 'pvalue', output_folder=target_gene)
# plot.histogram_of_column_and_save(volcano_plot_df, 'adjusted_pvalue', output_folder=target_gene)

# # calculate significant genes determined by cutoffs
# significant_genes_positive, significant_genes_negative  = plot.volcano_plot(input_file_path=volcano_input_filename, 
#                                                                         yaxis='pvalue', 
#                                                                         xaxis='logFC', 
#                                                                         output_folder=target_gene,
#                                                                         significance_threshold=0.05, 
#                                                                         logfold_positive_threshold=2, 
#                                                                         logfold_negative_threshold=-2)

# # create box plots of significant genes (uoregulated and downregulated genes)
# plot.create_gene_expression_boxplot(expression_df, significant_genes_positive, mutated_status_df, positive=1, output_folder=target_gene)
# plot.create_gene_expression_boxplot(expression_df, significant_genes_negative, mutated_status_df, positive=0, output_folder=target_gene)

# # Reset warnings to default behavior after the code block
# warnings.resetwarnings()